# 自動更新景點熱門度

In [1]:
##################################################################

#評論部分

##################################################################



#　把資料庫裡的東西倒進DicA裡 (KEY:景點編號   VALUE:景點提及次數)
sql ='SELECT a.A_id, a.A_content, b.Attr_id, c.Attr_name\
     FROM  `iiimap_art_to_att` AS b\
     LEFT JOIN  `iiimap_article` AS a ON a.A_id = b.Article_id\
     LEFT JOIN  `iiimap_attraction` AS c ON b.Attr_id = c.Attr_id\
     WHERE a.attr_in_art_count >=1;'

import MySQLdb
db = MySQLdb.connect('10.120.26.4','iii','iii','iii',charset='utf8')#連結資料庫
cursor = db.cursor()
DicA={}
cursor.execute(sql)  #執行sql指令
results = cursor.fetchall() #看結果

for row in results:
    if not str(row[2]) in DicA:
        DicA[str(row[2])] = 1.0
    else:
        DicA[str(row[2])] += 1.0

#####　檢視DicA的長度　###### ( 查看景點數量 )
print 'Attrs :',len(DicA)

############################


#####################################################################


#　把出現次數轉換成權重
import math
DicARe ={}
DicALog={}
ASum = sum(DicA.values())#　出現次數加總
print 'Total times for Attr:',ASum

for i in DicA:
    DicALog[i] = math.log(DicA[i]/ASum,10)
    
AMax = max(DicALog.values())#　拿出取Log之後的最大值

for i in DicALog:
    #print i,'\t',DicA[i],'\t',DicALog[i],'\t',AMax/DicALog[i]
    DicARe[i] = AMax/DicALog[i]


#####　檢視結果　###### ( KEY:Attr_id  VALUE:AWEIGHT )
#for i in DicARe:
#    if i in DicA:
#        print i,'\t',DicA[i],'\t',DicARe[i]
######################


#####################################################################


#　把資料分成三個等級
import operator
HOTDicA = {}
for i1 in DicARe:
    HOTDicA[i1]={}

DRLen = float(len(DicARe))#　算景點數量
DR1 = int(float(len(DicARe))/3.0)#　算1/3的景點數量
DR2 = int((float(len(DicARe))/3.0)*2.0)#　算2/3的景點數量

###　檢視有沒有分配平均　###
print 'Attr_Nums',DRLen,'\n',\
      '1/3_Of_Attr',DR1,'\n',\
      '2/3_Of_Attr',DR2

#　根據權重排序，決定等級
cnt2 = 0   
for i3 in sorted(DicARe.iteritems(),key=operator.itemgetter(1),reverse=True):
    cnt2 += 1
    HOTDicA[i3[0]]['AWEIGHT']=i3[1]
    if cnt2 <= DR1:
        HOTDicA[i3[0]]['AWLevel']=1
    elif cnt2 > DR1 and cnt2 <= DR2:
        HOTDicA[i3[0]]['AWLevel']=2
    else:
        HOTDicA[i3[0]]['AWLevel']=3


print 'count for DicAWEIGHT','\t',cnt2#　(讀取)景點數量
print 'count for HOTDicA Len','\t',len(HOTDicA)#　景點數量　

###　　檢查有沒有平均分配到三個等級　　###　
n1=0
n2=0
n3=0
for i in HOTDicA:
    if HOTDicA[i]['AWLevel']==1:
        n1+=1
    elif HOTDicA[i]['AWLevel']==2:
        n2+=1
    elif HOTDicA[i]['AWLevel']==3:
        n3+=1

print 'AWLevel 1 :',n1,'個\n'\
      'AWLevel 2 :',n2,'個\n'\
      'AWLevel 3 :',n3,'個\n'
        
        
        
##################################################################

#評論部分

##################################################################



#　把資料庫裡的東西倒進Dicc裡 (KEY:景點編號   VALUE:景點評論數)
sql ='SELECT a.Attr_id, a.Attr_name, COUNT(*)\
      FROM  `iiimap_attraction` AS a\
      JOIN  `iiimap_comment` AS c ON a.Attr_id = c.Attr_id\
      GROUP BY a.Attr_id;'

import MySQLdb
db = MySQLdb.connect('10.120.26.4','iii','iii','iii',charset='utf8')#連結資料庫
cursor = db.cursor()
DicC={}
cursor.execute(sql)  #執行sql指令
results = cursor.fetchall() #看結果

for row in results:
    Attr = str(row[0])
    Comments = float(row[2])
    DicC[Attr] = Comments
    
#####　檢視DicC的長度　###### ( 查看景點數量 )
print 'Attrs :',len(DicC)

############################


#####################################################################


#　把出現次數轉換成權重
import math
DicCRe ={}
DicCLog={}
CSum = sum(DicC.values())#　出現次數加總
print 'Total Comments:',CSum

for i in DicC:
    DicCLog[i] = math.log(DicC[i]/CSum,10)
    
CMax = max(DicCLog.values())#　拿出取Log之後的最大值

for i in DicCLog:
    #print i,'\t',DicC[i],'\t',DicCLog[i],'\t',CMax/DicCLog[i]
    DicCRe[i] = CMax/DicCLog[i]
    
#####　檢視結果　###### ( KEY:Attr_id  VALUE:AWEIGHT )
#for i in DicCRe:
#    if i in DicC:
#        print i,'\t',DicC[i],'\t',DicCRe[i]
######################


#####################################################################


#　把資料分成三個等級
import operator
HOTDicC = {}
for i1 in DicCRe:
    HOTDicC[i1]={}

DRLen = float(len(DicCRe))#　算景點數量
DR1 = int(float(len(DicCRe))/3.0)#　算1/3的景點數量
DR2 = int((float(len(DicCRe))/3.0)*2.0)#　算2/3的景點數量

###　檢視有沒有分配平均　###
print 'Attr_Nums',DRLen,'\n',\
      '1/3_Of_Attr',DR1,'\n',\
      '2/3_Of_Attr',DR2

#　根據權重排序，決定等級
cnt2 = 0   
for i3 in sorted(DicCRe.iteritems(),key=operator.itemgetter(1),reverse=True):
    cnt2 += 1
    HOTDicC[i3[0]]['CWEIGHT']=i3[1]
    if cnt2 <= DR1:
        HOTDicC[i3[0]]['CWLevel']=1
    elif cnt2 > DR1 and cnt2 <= DR2:
        HOTDicC[i3[0]]['CWLevel']=2
    else:
        HOTDicC[i3[0]]['CWLevel']=3

print 'count for DicCWEIGHT','\t',cnt2#　(讀取)景點數量
print 'count for HOTDicC Len','\t',len(HOTDicC)#　景點數量

###　　檢查有沒有平均分配到三個等級　　###　
n1=0
n2=0
n3=0
for i in HOTDicC:
    if HOTDicC[i]['CWLevel']==1:
        n1+=1
    elif HOTDicC[i]['CWLevel']==2:
        n2+=1
    elif HOTDicC[i]['CWLevel']==3:
        n3+=1

print 'CWLevel 1 :',n1,'個\n'\
      'CWLevel 2 :',n2,'個\n'\
      'CWLevel 3 :',n3,'個\n'
        
            
        
##################################################################

#結合部分

##################################################################



#　把兩個結果合併起來( KEY:景點編號  VALUE:評論&遊記的權重加總 )
DicF = {}
print 'Attr in Articles:',len(HOTDicA)
for i in HOTDicA:
    DicF[i]={}
    
print 'Attr in Comments:',len(HOTDicC)
for i in HOTDicC:
    DicF[i]={}

print 'Total Attrs :',len(DicF)
c1=0
c2=0
c3=0
for i in DicF:
    if i in HOTDicA and i in HOTDicC:
        c1 +=1
        #print HOTDicA[i],HOTDicC[i]
        DicF[i]=HOTDicA[i]['AWEIGHT']+HOTDicC[i]['CWEIGHT']
        
    elif i in HOTDicA and i not in HOTDicC:
        #print HOTDicA[i]
        DicF[i]=HOTDicA[i]['AWEIGHT']+0.0
        c2 +=1
        
    elif i in HOTDicC and i not in HOTDicA:
        #print HOTDicC[i]
        DicF[i]=HOTDicC[i]['CWEIGHT']+0.0
        c3 +=1
        
print 'Attr in Articles & Comments :','\t',c1,'\n',\
      'Attr Only in Articles :','\t',c2,'\n',\
      'Attr Only in Comments :','\t',c3

if (c1+c2+c3)==len(DicF):
    print 'Combine Successfully!'


#####################################################################


#　把資料分成三個等級
import operator
HOTDicFinal = {}
for i1 in DicF:
    HOTDicFinal[i1]={}

DRLen = float(len(DicF))#　算景點數量
DR1 = int(float(len(DicF))/3.0)#　算1/3的景點數量
DR2 = int((float(len(DicF))/3.0)*2.0)#　算2/3的景點數量

###　檢視有沒有分配平均　###
print 'Attr_Nums',DRLen,'\n',\
      '1/3_Of_Attr',DR1,'\n',\
      '2/3_Of_Attr',DR2

#　根據權重排序，決定等級
cnt2 = 0   
for i3 in sorted(DicF.iteritems(),key=operator.itemgetter(1),reverse=True):
    cnt2 += 1
    HOTDicFinal[i3[0]]['TWEIGHT']=i3[1]
    if cnt2 <= DR1:
        HOTDicFinal[i3[0]]['TWLevel']=1
    elif cnt2 > DR1 and cnt2 <= DR2:
        HOTDicFinal[i3[0]]['TWLevel']=2
    else:
        HOTDicFinal[i3[0]]['TWLevel']=3

print 'count for DicFWEIGHT','\t',cnt2#　(讀取)景點數量
print 'count for HOTDicFinal Len','\t',len(HOTDicFinal)#　景點數量

###　　檢查有沒有平均分配到三個等級　　###　
n1=0
n2=0
n3=0
for i in HOTDicFinal:
    if HOTDicFinal[i]['TWLevel']==1:
        n1+=1
    elif HOTDicFinal[i]['TWLevel']==2:
        n2+=1
    elif HOTDicFinal[i]['TWLevel']==3:
        n3+=1

print 'TWLevel 1 :',n1,'個\n'\
      'TWLevel 2 :',n2,'個\n'\
      'TWLevel 3 :',n3,'個\n'

Attrs : 654
Total times for Attr: 31690.0
Attr_Nums 654.0 
1/3_Of_Attr 218 
2/3_Of_Attr 436
count for DicAWEIGHT 	654
count for HOTDicA Len 	654
AWLevel 1 : 218 個
AWLevel 2 : 218 個
AWLevel 3 : 218 個

Attrs : 226
Total Comments: 52660.0
Attr_Nums 226.0 
1/3_Of_Attr 75 
2/3_Of_Attr 150
count for DicCWEIGHT 	226
count for HOTDicC Len 	226
CWLevel 1 : 75 個
CWLevel 2 : 75 個
CWLevel 3 : 76 個

Attr in Articles: 654
Attr in Comments: 226
Total Attrs : 678
Attr in Articles & Comments : 	202 
Attr Only in Articles : 	452 
Attr Only in Comments : 	24
Combine Successfully!
Attr_Nums 678.0 
1/3_Of_Attr 226 
2/3_Of_Attr 452
count for DicFWEIGHT 	678
count for HOTDicFinal Len 	678
TWLevel 1 : 226 個
TWLevel 2 : 226 個
TWLevel 3 : 226 個



In [2]:
#####　檢視結果　######


PrintDic = 'DF'


######################################################################


if PrintDic == 'DA':
    print '>Result Of DicA'
    print 'Attr\tTimes\tAWEIGHT\t\tAWLevel'
    print '---------------------------------------'
    for i in HOTDicA:
        print i,'\t',DicA[i],'\t',DicARe[i],'\t',HOTDicA[i]['AWLevel']
    
elif PrintDic == 'DC':
    print '>Result Of DicC'
    print 'Attr\tComms\tCWEIGHT\t\tCWLevel'
    print '---------------------------------------'
    for i in HOTDicC:
        print i,'\t',DicC[i],'\t',DicCRe[i],'\t',HOTDicC[i]['CWLevel']
    
elif PrintDic == 'DF':
    print '>Result Of DicF'
    print 'Attr\tTWEIGHT\t\tTWLevel'
    print '---------------------------------------'
    for i in HOTDicFinal:
        print i,'\t',HOTDicFinal[i]['TWEIGHT'],'\t',HOTDicFinal[i]['TWLevel']

>Result Of DicF
Attr	TWEIGHT		TWLevel
---------------------------------------
164 	0.390198921979 	2
1948 	0.348489194773 	2
1949 	0.308899702418 	3
428 	0.420404066932 	2
1943 	0.341231394839 	2
1945 	0.308899702418 	3
818 	0.449197819636 	2
342 	0.609132076297 	1
343 	0.288239919569 	3
813 	0.438758687011 	2
348 	0.332749756656 	2
1798 	0.288239919569 	3
1792 	0.413273374501 	2
1791 	0.322417900194 	2
1797 	0.332749756656 	2
1794 	0.288239919569 	3
1494 	0.288239919569 	3
914 	0.63160809896 	1
1498 	0.476906209515 	1
1701 	0.5414932275 	1
1702 	0.15448830932 	3
1703 	0.453248788119 	2
1704 	0.473908570968 	1
1705 	0.16500886855 	3
1707 	0.15448830932 	3
1708 	0.473213956214 	1
1060 	0.375006324979 	2
1062 	0.288239919569 	3
1067 	0.47391714751 	1
1066 	1.05760736793 	1
1391 	0.408136742213 	2
298 	0.288239919569 	3
299 	0.420404066932 	2
1667 	0.288239919569 	3
297 	0.554893941113 	1
295 	0.689716478162 	1
1662 	0.44272822889 	2
290 	0.370572487926 	2
291 	0.806867921252 	1
1128 	0.4

In [3]:
def ExportHot(HOTDicFinal,ExportType):
    import time
    Date = time.strftime('%Y%m%d',time.localtime(time.time()))
    JsonFile   ='AttrHot-'+Date+'.json'
    OutputFile ='AttrHot-'+Date+'.txt'
    
    if ExportType=='JSON':
        import json
        json_str = json.dumps(HOTDicFinal)#　丟進一JSON格式的變數
        with open(JsonFile,'w') as jfid:#　寫進Json檔裡
            jfid.write(json_str)
    
    elif ExportType=='TXT':
        with open(OutputFile,'w') as f:
            for i in HOTDicFinal:
                f.write(i+','+str(HOTDicFinal[i]['TWLevel'])+'\n')

# 把結果存出去
ExportHot(HOTDicFinal,ExportType)

HOTDicFinal : [字典]。存熱門度結果的字典 (KEY: 景點編號  VALUE: {熱門權重,熱門等級})

ExportType  : [字串]。['JSON'or'TXT']。

            'JSON' : 格式與 HOTDicFinal 一致
            'TXT'  : 景點編號 , 熱門等級 <---每行
            
結果將存成一　'AttrHot-YYYYMMDD'　的檔案。

＊熱門等級［1--熱門］。［2--普通］。［3--冷門］

In [4]:
ExportHot(HOTDicFinal,'JSON')